In [1]:
import os, argparse, json, re
from collections import defaultdict
import customutils
import pandas as pd
import numpy as np

LABELS = customutils.get_classes('configs/halpe26_classes.txt')
LABELS_COCO = customutils.get_classes('configs/coco_classes.txt')
LABELS_FOOT = ['LBigToe','RBigToe','LSmallToe','RSmallToe','LHeel','RHeel']

df_running_annotations = customutils.load_images_dataframe()
df_running_annotations

,image_id,pose_id,file_name,frame,person
0,1000,1000,Athletics_Mixed_Tokyo_2020_20_1.mp4,0,0
1,1001,1001,Athletics_Mixed_Tokyo_2020_20_1.mp4,1,0
2,1002,1002,Athletics_Mixed_Tokyo_2020_20_1.mp4,2,0
3,1003,1003,Athletics_Mixed_Tokyo_2020_20_1.mp4,3,0
4,1004,1004,Athletics_Mixed_Tokyo_2020_20_1.mp4,4,0
...,...,...,...,...,...
12198,46160,146160,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,160,1
12199,46161,146161,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,161,1
12200,46162,146162,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,162,1
12201,46163,146163,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,163,1


In [2]:
df = customutils.load_keypoints_dataframe()
df[df.duplicated(subset=['id'],keep=False)]

,id,image_id,category_id,bbox,area,iscrowd,num_keypoints,keypoints


In [3]:

df = df.drop(columns=['iscrowd', 'num_keypoints'])
df = df.set_index(['id'])
df

,image_id,category_id,bbox,area,keypoints
id,,,,,
1000,1000,1,"[957.5, 204.65, 294.5, 641.9200000000001]",189045.4400,"[1132.72, 271.76, 1, 1151.88, 256.58, 1, 1137...."
1001,1001,1,"[895.29, 201.46, 396.59000000000015, 639.68999...",253694.6571,"[1132.72, 268.57, 1, 1151.88, 253.39, 1, 1125...."
1002,1002,1,"[882.53, 207.84, 459.21000000000004, 641.29]",294486.7809,"[1132.72, 255.81, 1, 1148.69, 247.01, 1, 1132...."
1003,1003,1,"[879.33, 187.1, 459.2199999999999, 658.84]",302552.5048,"[1129.53, 249.42, 1, 1145.5, 240.63, 1, 1129.1..."
1004,1004,1,"[912.84, 201.46, 406.57000000000005, 681.17]",276943.2869,"[1132.72, 249.42, 1, 1148.69, 240.63, 1, 1132...."
...,...,...,...,...,...
146160,46160,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3..."
146161,46161,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3..."
146162,46162,1,"[1043.37, 309.11, 277.4100000000001, 700.94999...",194450.5395,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3..."


In [4]:

df_categories = customutils.load_categories_dataframe()
kpts_classes = df_categories[df_categories['id'] == 1]['keypoints'][0]

In [5]:

df['dist_alphapose'] = np.ones((len(df),1)) * np.inf
df['dist_openpose'] = np.ones((len(df),1)) * np.inf
df['dist_vitpose'] = np.ones((len(df),1)) * np.inf
df['conf_alphapose'] = float(0.0)
df['conf_openpose'] = float(0.0)
df['conf_vitpose'] = float(0.0)
df['oks_alphapose'] = float(0.0)
df['oks_openpose'] = float(0.0)
df['oks_vitpose'] = float(0.0)
df['oks_coco_alphapose'] = float(0.0)
df['oks_coco_openpose'] = float(0.0)
df['oks_coco_vitpose'] = float(0.0)
df['oks_foot_alphapose'] = float(0.0)
df['oks_foot_openpose'] = float(0.0)
df['oks_foot_vitpose'] = float(0.0)
df['kpts_alphapose'] = ""#np.zeros((len(df),78)).tolist()
df['kpts_openpose'] = ""#np.zeros((len(df),78)).tolist()
df['kpts_vitpose'] = ""#np.zeros((len(df),78)).tolist()
df

,image_id,category_id,bbox,area,keypoints,dist_alphapose,dist_openpose,dist_vitpose,conf_alphapose,conf_openpose,...,oks_vitpose,oks_coco_alphapose,oks_coco_openpose,oks_coco_vitpose,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose,kpts_alphapose,kpts_openpose,kpts_vitpose
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5, 204.65, 294.5, 641.9200000000001]",189045.4400,"[1132.72, 271.76, 1, 1151.88, 256.58, 1, 1137....",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,
1001,1001,1,"[895.29, 201.46, 396.59000000000015, 639.68999...",253694.6571,"[1132.72, 268.57, 1, 1151.88, 253.39, 1, 1125....",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,
1002,1002,1,"[882.53, 207.84, 459.21000000000004, 641.29]",294486.7809,"[1132.72, 255.81, 1, 1148.69, 247.01, 1, 1132....",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,
1003,1003,1,"[879.33, 187.1, 459.2199999999999, 658.84]",302552.5048,"[1129.53, 249.42, 1, 1145.5, 240.63, 1, 1129.1...",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,
1004,1004,1,"[912.84, 201.46, 406.57000000000005, 681.17]",276943.2869,"[1132.72, 249.42, 1, 1148.69, 240.63, 1, 1132....",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146160,46160,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,
146161,46161,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,
146162,46162,1,"[1043.37, 309.11, 277.4100000000001, 700.94999...",194450.5395,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",inf,inf,inf,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,


In [6]:

def edit_keypoints(kpts):
    kpts = np.array(kpts).reshape(-1,3).astype(float)
    vi = kpts[:,2]
    kpts = kpts[:,0:2]
    return kpts, vi

def get_keypoint(kpts, id):
    index = kpts_classes.index(id)
    return [kpts[index*3],kpts[(index*3)+1]]

def load_keypoints(root, model):
    dist = 'dist_' + str.lower(model)
    kpts = 'kpts_' + str.lower(model)
    conf = 'conf_' + str.lower(model)
    oks = 'oks_' + str.lower(model)
    oks_coco = 'oks_coco_' + str.lower(model)
    oks_foot = 'oks_foot_' + str.lower(model)
    file = os.path.join(root, model, 'person_keypoints_running.json')
    with open(file, 'r') as f:
        predictions = json.load(f)
        d = {}
        
        for prediction in predictions:
            # if prediction['image_id'] != 46162:
            #     continue
            pose_id = -1
            mask_frame = df['image_id'] == prediction['image_id']
            gts = df[mask_frame]
            dt = prediction['keypoints']
            kpts1, vi1 = edit_keypoints(dt)
            min_dist = np.inf #np.ones((len(kpts1),1)) * 
            
            # for i in range(len(gts)):
            for pose_id in gts.index:
                gt = gts['keypoints'][pose_id]
                kpts2, vi2 = edit_keypoints(gt)
                d = np.linalg.norm(kpts1 - kpts2, ord=2, axis=1)
                v = np.ones(len(d))

                for part in range(len(d)):
                    if vi1[part] == 0 or vi2[part] == 0:
                        d[part] = 0
                        v[part] = 0
                                        
                area = customutils.compute_area_keypoints(gt)

                # compute head size for distance normalization
                head = get_keypoint(gt,"head")
                neck = get_keypoint(gt,"neck")

                headSize = 1
                if (len(head) > 0 and len(neck) > 0):
                    headSize = customutils.get_head_size(head[0], head[1], neck[0], neck[1])
                # normalize distance
                dNorm = np.sum(d)/headSize

                oks_val = customutils.OKS(gt, dt, area, 0, 26)
                oks_val_coco = customutils.OKS(gt, dt, area, 0, 17)
                oks_val_foot = customutils.OKS(gt, dt, area, 20, 26)

                if dNorm < min_dist and dNorm < gts[dist][pose_id]:
                    min_dist = dNorm
                    df.loc[pose_id, dist] = min_dist
                    df.loc[pose_id, oks] = oks_val
                    df.loc[pose_id, oks_coco] = oks_val_coco
                    df.loc[pose_id, oks_foot] = oks_val_foot
                    df.loc[pose_id, kpts] = ','.join(str(element) for element in dt)
                    df.loc[pose_id, conf] = prediction['score']
            
        # df_running_annotations['score_' + str.lower(model)] = df_running_annotations['image_id'].map(d)

In [10]:
root = '../videos/results/'
load_keypoints(root, 'openpose')
load_keypoints(root, 'alphapose')
load_keypoints(root, 'ViTPose')

In [ ]:
# remove unmatched poses/frames
df = df.loc[(df['conf_openpose'] > float(0)) & (df['conf_alphapose'] > float(0)) & (df['conf_vitpose'] > float(0))]
df

,image_id,category_id,bbox,area,keypoints,dist_alphapose,dist_openpose,dist_vitpose,conf_alphapose,conf_openpose,...,oks_vitpose,oks_coco_alphapose,oks_coco_openpose,oks_coco_vitpose,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose,kpts_alphapose,kpts_openpose,kpts_vitpose
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5, 204.65, 294.5, 641.9200000000001]",189045.4400,"[1132.72, 271.76, 1, 1151.88, 256.58, 1, 1137....",7.334525,7.013630,6.367525,0.877560,0.683721,...,0.669911,0.867149,0.854623,0.837240,0.094100,0.102082,0.195812,"1136.07421875,266.266845703125,2,1155.60852050...","1139.55,268.183,2,1160.02,256.521,2,1139.39,24...","1135.575439453125,264.9145202636719,2,1153.360..."
1001,1001,1,"[895.29, 201.46, 396.59000000000015, 639.68999...",253694.6571,"[1132.72, 268.57, 1, 1151.88, 253.39, 1, 1125....",6.929502,7.620146,8.830845,0.867543,0.711724,...,0.665877,0.815075,0.832625,0.820795,0.637555,0.283690,0.226942,"1134.0335693359375,257.7659912109375,2,1153.15...","1139.57,259.547,2,1157.16,247.864,2,1139.4,244...","1133.6328125,259.5325622558594,2,1151.58813476..."
1002,1002,1,"[882.53, 207.84, 459.21000000000004, 641.29]",294486.7809,"[1132.72, 255.81, 1, 1148.69, 247.01, 1, 1132....",6.324701,5.498150,4.783507,0.869874,0.753456,...,0.819871,0.872357,0.903906,0.910690,0.776696,0.725340,0.562553,"1128.94189453125,252.69618225097656,2,1148.562...","1139.6,247.833,2,1157.25,244.735,2,1139.4,235....","1136.784423828125,251.25259399414062,2,1154.65..."
1003,1003,1,"[879.33, 187.1, 459.2199999999999, 658.84]",302552.5048,"[1129.53, 249.42, 1, 1145.5, 240.63, 1, 1129.1...",4.901953,5.589828,4.732465,0.887327,0.755095,...,0.850528,0.907709,0.905083,0.918811,0.781138,0.695457,0.657061,"1129.5250244140625,242.39736938476562,2,1143.6...","1136.53,244.883,2,1148.4,241.765,2,1133.67,227...","1132.8712158203125,246.05426025390625,2,1151.2..."
1004,1004,1,"[912.84, 201.46, 406.57000000000005, 681.17]",276943.2869,"[1132.72, 249.42, 1, 1148.69, 240.63, 1, 1132....",5.010580,5.075816,5.901758,0.891673,0.756369,...,0.745938,0.922700,0.956517,0.943024,0.677517,0.578462,0.187528,"1127.0386962890625,248.37823486328125,2,1148.4...","1133.7,244.898,2,1148.37,241.799,2,1133.6,227....","1129.1875,247.21041870117188,2,1149.0015869140..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146159,46159,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.501191,7.240335,6.082551,0.868441,0.675163,...,0.714262,0.764059,0.764995,0.891781,0.405810,0.377975,0.211289,"1141.66064453125,357.68817138671875,2,1148.506...","1118.98,374.317,2,1133.63,362.501,2,1115.9,362...","1122.9000244140625,359.6590270996094,2,1136.55..."
146160,46160,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.738347,7.812317,6.460968,0.875172,0.663894,...,0.698358,0.732880,0.732826,0.870795,0.540675,0.362869,0.209784,"1143.8115234375,357.8825988769531,2,1150.61694...","1118.99,377.24,2,1133.62,362.577,2,1115.92,365...","1124.10986328125,359.1108093261719,2,1138.1390..."
146161,46161,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.503957,7.600723,6.263417,0.875935,0.673371,...,0.707576,0.803033,0.740115,0.884123,0.509892,0.408483,0.207361,"1132.1380615234375,357.40948486328125,2,1138.9...","1118.97,377.231,2,1133.62,362.538,2,1115.89,36...","1122.510498046875,360.0570373535156,2,1136.302..."


In [9]:
df[['image_id','oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']]
df2 = df[['oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']].mean()
df2

oks_alphapose         0.716473
oks_openpose          0.000000
oks_vitpose           0.000000
oks_coco_alphapose    0.753881
oks_coco_openpose     0.000000
oks_coco_vitpose      0.000000
oks_foot_alphapose    0.652181
oks_foot_openpose     0.000000
oks_foot_vitpose      0.000000
dtype: float64

In [ ]:
df[df['dist'] == np.inf]['image_id']

,id,image_id,category_id,bbox,area,keypoints,dist
7390,23110,23110,1,"[1287.97, 414.4, 197.62999999999988, 450.03]",88939.4289,"[1447.58, 440.42, 1, 1448.41, 431.89, 1, 1444....",inf
7501,123110,23110,1,"[866.8, 422.81, 310.06999999999994, 422.390000...",130970.4673,"[1067.9, 458.18, 1, 1068.63, 446.11, 1, 1061.8...",inf
7612,223110,23110,1,"[697.52, 423.99, 279.68000000000006, 500.85]",140077.7280,"[917.34, 473.76, 1, 914.43, 459.42, 1, 911.74,...",inf
8385,28086,28086,1,"[939.08, 284.98, 253.93999999999994, 624.89]",158684.5666,"[1167.35, 332.35, 1, 1164.79, 318.48, 1, 1163....",inf
8472,128086,28086,1,"[1159.41, 310.21, 307.77, 563.01]",173277.5877,"[1379.47, 344.19, 1, 1376.62, 335.7, 1, 1373.4...",inf
10356,39068,39068,1,"[58.07, 41.54, 534.3299999999999, 1015.3400000...",542526.6222,"[237.94, 93.4, 1, 261.62, 77.12, 1, 230.46, 80...",inf
10425,139068,39068,1,"[736.28, 98.54, 338.8299999999999, 889.7900000...",301487.5457,"[838.56, 149.8, 1, 855.92, 135.36, 1, 820.05, ...",inf
10749,41074,41074,1,"[934.73, 273.97, 296.0999999999999, 853.899999...",252839.7900,"[1044.95, 329.82, 1, 1064.89, 312.61, 1, 1033....",inf
10824,141074,41074,1,"[1024.74, 226.63, 211.23000000000002, 809.7800...",171049.8294,"[1094.91, 286.82, 1, 1106.16, 270.88, 1, 1093....",inf
11258,143068,43068,1,"[1118.29, 299.05, 197.46000000000004, 673.4200...",132973.5132,"[1249.95, 326.29, 1, 1223.89, 323.66, 1, 1242....",inf


In [ ]:
def load_results(root, model):
    file = os.path.join(root, 'results_pckh_' + model + '.json')
    with open(file, 'r') as f:
        predictions = json.load(f)
        d = {}
        for prediction in predictions:
            d[prediction['image_id']] = prediction['score']
            
        df_running_annotations['score_' + str.lower(model)] = df_running_annotations['image_id'].map(d)

In [11]:
# remove unmatched poses/frames
df_excl_hidden = df.loc[(df['oks_foot_alphapose'] > 0.01) | (df['oks_foot_openpose'] > 0.01) | (df['oks_foot_vitpose'] > 0.01)]
df_excl_hidden

,image_id,category_id,bbox,area,keypoints,dist_alphapose,dist_openpose,dist_vitpose,conf_alphapose,conf_openpose,...,oks_vitpose,oks_coco_alphapose,oks_coco_openpose,oks_coco_vitpose,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose,kpts_alphapose,kpts_openpose,kpts_vitpose
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5, 204.65, 294.5, 641.9200000000001]",189045.4400,"[1132.72, 271.76, 1, 1151.88, 256.58, 1, 1137....",7.334525,7.013630,6.367525,0.877560,0.683721,...,0.839584,0.867149,0.854623,0.837240,0.734402,0.742966,0.846226,"1136.07421875,266.266845703125,2,1155.60852050...","1139.55,268.183,2,1160.02,256.521,2,1139.39,24...","1135.575439453125,264.9145202636719,2,1153.360..."
1001,1001,1,"[895.29, 201.46, 396.59000000000015, 639.68999...",253694.6571,"[1132.72, 268.57, 1, 1151.88, 253.39, 1, 1125....",6.929502,7.620146,8.830845,0.867543,0.711724,...,0.814032,0.815075,0.832625,0.820795,0.963230,0.866944,0.794871,"1134.0335693359375,257.7659912109375,2,1153.15...","1139.57,259.547,2,1157.16,247.864,2,1139.4,244...","1133.6328125,259.5325622558594,2,1151.58813476..."
1002,1002,1,"[882.53, 207.84, 459.21000000000004, 641.29]",294486.7809,"[1132.72, 255.81, 1, 1148.69, 247.01, 1, 1132....",6.324701,5.498150,4.783507,0.869874,0.753456,...,0.922007,0.872357,0.903906,0.910690,0.978284,0.971253,0.954072,"1128.94189453125,252.69618225097656,2,1148.562...","1139.6,247.833,2,1157.25,244.735,2,1139.4,235....","1136.784423828125,251.25259399414062,2,1154.65..."
1003,1003,1,"[879.33, 187.1, 459.2199999999999, 658.84]",302552.5048,"[1129.53, 249.42, 1, 1145.5, 240.63, 1, 1129.1...",4.901953,5.589828,4.732465,0.887327,0.755095,...,0.929315,0.907709,0.905083,0.918811,0.977284,0.967039,0.959075,"1129.5250244140625,242.39736938476562,2,1143.6...","1136.53,244.883,2,1148.4,241.765,2,1133.67,227...","1132.8712158203125,246.05426025390625,2,1151.2..."
1004,1004,1,"[912.84, 201.46, 406.57000000000005, 681.17]",276943.2869,"[1132.72, 249.42, 1, 1148.69, 240.63, 1, 1132....",5.010580,5.075816,5.901758,0.891673,0.756369,...,0.919301,0.922700,0.956517,0.943024,0.968041,0.950892,0.852084,"1127.0386962890625,248.37823486328125,2,1148.4...","1133.7,244.898,2,1148.37,241.799,2,1133.6,227....","1129.1875,247.21041870117188,2,1149.0015869140..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146159,46159,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.501191,7.240335,6.082551,0.868441,0.675163,...,0.884817,0.764059,0.764995,0.891781,0.910295,0.886464,0.865086,"1141.66064453125,357.68817138671875,2,1148.506...","1118.98,374.317,2,1133.63,362.501,2,1115.9,362...","1122.9000244140625,359.6590270996094,2,1136.55..."
146160,46160,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.738347,7.812317,6.460968,0.875172,0.663894,...,0.869753,0.732880,0.732826,0.870795,0.944972,0.881181,0.866799,"1143.8115234375,357.8825988769531,2,1150.61694...","1118.99,377.24,2,1133.62,362.577,2,1115.92,365...","1124.10986328125,359.1108093261719,2,1138.1390..."
146161,46161,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.503957,7.600723,6.263417,0.875935,0.673371,...,0.879295,0.803033,0.740115,0.884123,0.938004,0.893016,0.865615,"1132.1380615234375,357.40948486328125,2,1138.9...","1118.97,377.231,2,1133.62,362.538,2,1115.89,36...","1122.510498046875,360.0570373535156,2,1136.302..."


In [59]:
df_oks = df_excl_hidden[['image_id','oks_alphapose','oks_openpose','oks_vitpose','oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']]#.sort_values(['oks_foot_alphapose'])

In [12]:
df_excl_hidden_mean = df_excl_hidden[['oks_alphapose','oks_openpose','oks_vitpose',	'oks_coco_alphapose',	'oks_coco_openpose',	'oks_coco_vitpose',	'oks_foot_alphapose',	'oks_foot_openpose',	'oks_foot_vitpose']].mean()
df_excl_hidden_mean

oks_alphapose         0.727442
oks_openpose          0.683418
oks_vitpose           0.748067
oks_coco_alphapose    0.763886
oks_coco_openpose     0.732074
oks_coco_vitpose      0.790271
oks_foot_alphapose    0.668824
oks_foot_openpose     0.604274
oks_foot_vitpose      0.628487
dtype: float64

In [13]:
df_oks[(df_oks['image_id'] >= 28000) & (df_oks['image_id'] <= 28086)]

NameError: name 'df_oks' is not defined

In [ ]:
df_running_annotations[df_running_annotations['image_id'] == 13039]

In [46]:
df_running_annotations[df_running_annotations['file_name'] == 'World_Athletics_Men_5000m_Oregon_2022_18.mp4']

,image_id,pose_id,file_name,frame,person
8299,28000,28000,World_Athletics_Men_5000m_Oregon_2022_18.mp4,0,0
8300,28001,28001,World_Athletics_Men_5000m_Oregon_2022_18.mp4,1,0
8301,28002,28002,World_Athletics_Men_5000m_Oregon_2022_18.mp4,2,0
8302,28003,28003,World_Athletics_Men_5000m_Oregon_2022_18.mp4,3,0
8303,28004,28004,World_Athletics_Men_5000m_Oregon_2022_18.mp4,4,0
...,...,...,...,...,...
8468,28082,128082,World_Athletics_Men_5000m_Oregon_2022_18.mp4,82,1
8469,28083,128083,World_Athletics_Men_5000m_Oregon_2022_18.mp4,83,1
8470,28084,128084,World_Athletics_Men_5000m_Oregon_2022_18.mp4,84,1
8471,28085,128085,World_Athletics_Men_5000m_Oregon_2022_18.mp4,85,1
